## Code to Extract Exponential Smoothing Parameters

***

In [6]:
# load modules
import statsmodels.tsa.holtwinters as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

We need to read in each protected dataset, fit the SES model, and extract the $\alpha$ parameter for each series. Save these as a dataset, and plot in R.

In [3]:
# results file path
results_path = "../../Outputs/Results/"

In [5]:
train_data = pd.read_csv("../../Data/Train/Clean/m3_monthly_micro_h1.csv", header=None, skiprows=1)
# convert to a list of series, and drop missing values
train_data = [x.dropna() for _, x in train_data.iterrows()]

In [43]:
files = os.listdir("../../Data/Train/Clean/")

In [44]:
files = [f for f in files if ("protected_m3_monthly_micro_h1_" in f or "full_m3" in f)]

In [45]:
files

['full_m3_monthly_micro_clean.csv',
 'protected_m3_monthly_micro_h1_AN_0.5.csv',
 'protected_m3_monthly_micro_h1_AN_1.5.csv',
 'protected_m3_monthly_micro_h1_AN_1.csv',
 'protected_m3_monthly_micro_h1_AN_2.csv',
 'protected_m3_monthly_micro_h1_Bottom_0.1.csv',
 'protected_m3_monthly_micro_h1_Bottom_0.2.csv',
 'protected_m3_monthly_micro_h1_Bottom_0.4.csv',
 'protected_m3_monthly_micro_h1_DP_0.1.csv',
 'protected_m3_monthly_micro_h1_DP_1.csv',
 'protected_m3_monthly_micro_h1_DP_10.csv',
 'protected_m3_monthly_micro_h1_DP_20.csv',
 'protected_m3_monthly_micro_h1_DP_4.6.csv',
 'protected_m3_monthly_micro_h1_k-nts_10.csv',
 'protected_m3_monthly_micro_h1_k-nts_15.csv',
 'protected_m3_monthly_micro_h1_k-nts_5.csv',
 'protected_m3_monthly_micro_h1_Top_0.1.csv',
 'protected_m3_monthly_micro_h1_Top_0.2.csv',
 'protected_m3_monthly_micro_h1_Top_0.4.csv']

In [46]:
methods = {"full":[], "DP": [0.1, 1, 4.6, 10, 20], "Top": [0.1, 0.2, 0.4], "k-nts": [5, 10, 15]}

In [57]:
alpha_dict = {}
for method in methods.items():
    method_files = [f for f in files if method[0] in f]
    
    if method[0] == "full":
        param_files = method_files
        [param_files] = param_files
        
        full_data = pd.read_csv("../../Data/Train/Clean/"+param_files, header=None, skiprows=1)
        full_data = [x.dropna() for _, x in full_data.iterrows()]
        full_data = [pd.Series([i if i >= 1 else 1 for i in x]) for x in full_data]
        full_data = [np.log(x) for x in full_data]
        
        alpha = []
        for x in full_data:
            ES = sm.ExponentialSmoothing(endog=x, initialization_method = 'estimated')
            ses = ES.fit()
            alpha.append(ses.params['smoothing_level'])
            
        alpha_dict["original"] = alpha
    
    else: 
        for param in method[1]:
            param_files = [f for f in method_files if "_"+str(param)+"." in f]
            [param_files] = param_files
            
            full_data = pd.read_csv("../../Data/Train/Clean/"+param_files, header=None, skiprows=1)
            full_data = [x.dropna() for _, x in full_data.iterrows()]
            full_data = [pd.Series([i if i >= 1 else 1 for i in x]) for x in full_data]
            full_data = [np.log(x) for x in full_data]
            
            alpha = []
            for x in full_data:
                ES = sm.ExponentialSmoothing(endog=x, initialization_method = 'estimated')
                ses = ES.fit()
                alpha.append(ses.params['smoothing_level'])
            
            alpha_dict[method[0]+"_"+str(param)] = alpha
    

        
#         ES = sm.ExponentialSmoothing(endog = x,                           # time series to model
#                              initialization_method = 'estimated') 

In [62]:
alpha_frame = pd.DataFrame(alpha_dict)

In [63]:
alpha_frame.to_csv("../../Outputs/Results/Model Parameters/ses_alphas.csv", index=False)